# Bring Your Own Model on Krutrim Cloud
### This notebook demonstrates how to deploy a model to Krutrim Cloud, monitor the status of the deployment, and manage tasks such as cancellation. 

### Install Krutrim Cloud SDK

In [ ]:
%pip install krutrim-cloud

### Prerequisite
1. Create a S3 Bucket on cloud.olakrutrim.com -> Storage
2. Export the required environment variables or store them in .env in examples directory

    KRUTRIM_CLOUD_API_KEY="Your Model API Key" <- Go to https://cloud.olakrutrim.com/console/inference-service?section=api-keys 

    KRUTRIM_CLOUD_S3_PUBLIC_KEY="Your S3 Bucket Public Key" <- Go to https://cloud.olakrutrim.com/console/settings/api-keys?section=api-keys

    KRUTRIM_CLOUD_S3_SECRET_KEY="Your S3 Bucket Secret Key" <- Go to https://cloud.olakrutrim.com/console/settings/api-keys?section=api-keys

    KRUTRIM_CLOUD_S3_BUCKET_ENDPOINT="Your S3 Bucket Endpoint" <- Go to https://cloud.olakrutrim.com/console/storage?section=object_storage

    KRUTRIM_CLOUD_S3_REGION="Your S3 Bucket Region" i.e. "ap-south-1" <- Go to https://cloud.olakrutrim.com/console/storage?section=object_storage

### Importing Required Libraries and Environment Variables
In this section, we import the necessary libraries and load environment variables from the `.env` file.

In [ ]:
# Import necessary libraries
from krutrim_cloud import KrutrimCloud
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

### Krutrim Cloud Client Initialization
In this step, we create a client instance of model registry and deploy modules to communicate with the Krutrim Cloud API.


In [ ]:
# Initialize KrutrimCloud client
client = KrutrimCloud()

### Setting Model and S3 Configuration
This section defines the bucket name, model ID, and version for deployment.

In [ ]:
# Define your model bucket and version details
bucket_name = "<your-bucket-name>"  # "my-bucket-name"
model_name = "<your-model-name>"  # "model_name" 
version = "<model-version>"  # "v1" <- without '.' in version string
local_directory = "<local-dirpath-to-upload-files>"  # "./model"

### Uploading Files to S3
In this section, we upload files from the local directory to the S3 bucket for the model version.
This functionality allows you to transfer local files for a model to the S3 bucket where Krutrim Cloud can access them.

In [ ]:
try:
    client.registry.model_registry.upload_files_to_s3(
            local_dir_path=local_directory,
            bucket_name=bucket_name,
            model_id=model_name,
            version=version,
        )
except Exception as exc:
    print(f"Exception while uploading/registering the model: {exc}")

### Model Deployment

Now we deploy the model to Krutrim Cloud. The S3 endpoint, access key, and secret key are fetched from the environment variables. This section also handles the deployment response and retrieves the task ID and name.

In [ ]:
try:
    # Deploy the model using the KrutrimCloud client
    deploy_response = client.deploy.tasks.create(
        bucket_name=bucket_name,
        version=version,
        model=model_name,
        s3_endpoint=os.environ.get("KRUTRIM_CLOUD_S3_BUCKET_ENDPOINT"),  # type: ignore
        s3_access_key=os.environ.get("KRUTRIM_CLOUD_S3_PUBLIC_KEY"),  # type: ignore
        s3_secret=os.environ.get("KRUTRIM_CLOUD_S3_SECRET_KEY"),  # type: ignore
        s3_region=os.environ.get("KRUTRIM_CLOUD_S3_REGION")  # S3 region # type: ignore
    )

    # Extract model name and task ID from the response
    deploy_model_name = deploy_response.name
    deploy_task_id = deploy_response.id

    # Print deployed model details
    print(f"Deployed Model Name: {deploy_model_name}")
    print(f"Deploy Task ID: {deploy_task_id}")
except Exception as exc:
    # Print the exception if something goes wrong
    print(f"Exception: {exc}")

### Task Monitoring and Status Retrieval
After deployment, it's essential to monitor the task's status. Here, we retrieve the deployment task status and print it out.


In [ ]:
try:    
    # Retrieve the deployment task status
    retrieve_response = client.deploy.tasks.retrieve(id=deploy_task_id)
    print(f"Deploy Task Status: {retrieve_response.status}")
except Exception as exc:
    # Print the exception if something goes wrong
    print(f"Exception: {exc}")

### List All Tasks
This section lists all tasks currently available in Krutrim Cloud, displaying their status and IDs.

In [ ]:
try:
    # List all tasks and print their IDs and statuses
    task_list = client.deploy.tasks.list().task_list
    for task in task_list:
        print(f"Task ID: {task['id']} | Status: {task['status']} | Model Name: {task['name']}")
except Exception as exc:
    # Print the exception if something goes wrong
    print(f"Exception: {exc}")

### Run Inference on deployed model

In [ ]:
model_id = client.registry.model_registry.fetch_model_id(model_name)
infer_model_name = f"{model_id}:{deploy_task_id}"

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": "Give two random number.  Provide only number without any explanation, for example: \nRandom Number: <number>",
    },
]

response = client.chat.completions.create(model=infer_model_name, messages=messages)

# Access generated output
txt_output_data = response.choices[0].message.content  # type:ignore
print(f"Inference Output: \n{txt_output_data}")

### Task Cancellation
If needed, you can cancel a task. This section demonstrates how to cancel the deployment task by its ID.


In [ ]:
try:
    # Cancel the deployment task if necessary
    client.deploy.tasks.cancel(deploy_task_id)
except Exception as exc:
    # Print the exception if something goes wrong
    print(f"Exception: {exc}")